In [54]:
import numpy as np
from copy import deepcopy

## Dynamic programming

pitfalls
- Brute force algorithm
- requires looping through all possible states and action
- mot practicle for Lage state or action spaces.

In [63]:
LARGE = [
  [0  ,0   ,0  ,0  ,0  ,0  ,1  ],
  [0  ,'x' ,'x',0  ,'x',0  ,-1 ],
  [0  ,0   ,'x',0  ,'x',0  ,1  ],
  [0  ,-1  ,0  ,0  ,'x',0  ,'x'],
  [5  ,-1  ,0  ,0  ,0  ,0  ,0  ],
]

DEFAULT = [
  [0  ,0   ,0  ,0  ,1  ],
  [0  ,'x' ,'x',0  ,-1 ],
  [0  ,0   ,0  ,0  ,0  ],
]

SMALL = [
  [0  ,0   ,0  ,1  ],
  [0  ,'x' ,0  ,-1 ],
  [0  ,0   ,0  ,0  ],
]

In [56]:
class Gridworld:
    def __init__(self, architecture, walking_penality=-0.1):
        self.grid = deepcopy(architecture)
        self.available_states = []
        self.terminal_states = []
        self.available_moves = {}
        
        self.height = len(self.grid)
        self.width = len(self.grid[0])
        self.move_prob = 0.8

        for y in range(self.height):
            for x in range(self.width):

                # Check if state is a valid position to be in
                if (self.grid[y][x] != 'x'):

                    # Check if state is a terminal position
                    if (self.grid[y][x] > 0 or self.grid[y][x] < 0):
                        self.terminal_states.append((y,x))
                        
                    if (self.grid[y][x] == 0):
                        self.available_states.append((y,x))
                        # set move penalty
                        self.grid[y][x] = -0.1
                
                    
                    
        for y, x in self.available_states:
            self.available_moves[(y, x)] = self.get_valid_moves((y, x))

    def get_valid_moves(self, state):
        y = state[0]
        x = state[1]
        valid_moves = []
        
        # Action Up
        new_loc = (y - 1 if y - 1 > 0 else 0, x)
        if (self.grid[new_loc[0]][new_loc[1]] != 'x'):
            valid_moves.append(("UP", self.grid[new_loc[0]][new_loc[1]]))
                               
        # Action Right
        new_loc = (y, x + 1 if x + 1 < self.width else x)
        if (self.grid[new_loc[0]][new_loc[1]] != 'x'):
            valid_moves.append(("RIGHT", self.grid[new_loc[0]][new_loc[1]]))

        # Action Down
        new_loc = (y + 1 if y + 1 < self.height else y, x)
        if (self.grid[new_loc[0]][new_loc[1]] != 'x'):
            valid_moves.append(("DOWN", self.grid[new_loc[0]][new_loc[1]]))

                            
        # Action Left
        new_loc = (y, x - 1 if x - 1 > 0 else 0)
        if (self.grid[new_loc[0]][new_loc[1]] != 'x'):
            valid_moves.append(("LEFT", self.grid[new_loc[0]][new_loc[1]]))
        
        return valid_moves

    def reward_of_action(self, action, state):
        y = state[0]
        x = state[1]
        
        # returns (s', R(s'))
        if action == "UP":
            new_loc = (y - 1 if y - 1 > 0 else 0, x)
            if (self.grid[new_loc[0]][new_loc[1]] != 'x'):
                return new_loc , self.grid[new_loc[0]][new_loc[1]]
            else:
                return (y, x) , self.grid[y][x]

        elif action == "RIGHT":
            new_loc = (y, x + 1 if x + 1 < self.width else x)
            if (self.grid[new_loc[0]][new_loc[1]] != 'x'):
                return new_loc , self.grid[new_loc[0]][new_loc[1]]
            else:
                return (y, x) , self.grid[y][x]
        

        elif action == "DOWN":
            new_loc = (y + 1 if y + 1 < self.height else y, x)
            if (self.grid[new_loc[0]][new_loc[1]] != 'x'):
                return new_loc , self.grid[new_loc[0]][new_loc[1]]
            else:
                return (y, x) , self.grid[y][x]

        elif action == "LEFT":
            new_loc = (y, x - 1 if x - 1 > 0 else 0)
            if (self.grid[new_loc[0]][new_loc[1]] != 'x'):
                return new_loc , self.grid[new_loc[0]][new_loc[1]]
            else:
                return (y, x) , self.grid[y][x]
    
    def transition_probabilities(self, action, state):
        y = state[0]
        x = state[1]
        # returns list of (probability, reward, next_state)
        probs = []
    
        next_state, next_reward = self.reward_of_action(action, (y, x))
        probs.append((self.move_prob, next_reward, next_state))
        
        
        disobey_probs = 1 - self.move_prob
        
        
        if action == "UP" or action == "DOWN":
            next_state, next_reward = self.reward_of_action("LEFT", (y, x))
            probs.append((disobey_probs / 2, next_reward, next_state))
            next_state, next_reward = self.reward_of_action("RIGHT", (y, x))
            probs.append((disobey_probs / 2, next_reward, next_state))
        
        if action == "LEFT" or action == "RIGHT":
            next_state, next_reward = self.reward_of_action("UP", (y, x))
            probs.append((disobey_probs / 2, next_reward, next_state))
            
            next_state, next_reward = self.reward_of_action("DOWN", (y, x))
            probs.append((disobey_probs / 2, next_reward, next_state))
        
        return probs

g = Gridworld(DEFAULT)
g.get_valid_moves((0,0))
# g.reward_of_action("UP", (0 ,2))
# reward_of_action

[('UP', -0.1), ('RIGHT', -0.1), ('DOWN', -0.1), ('LEFT', -0.1)]

The Bellman equation is defined as follows:

\begin{equation*}
V(s) = max_a \left( R(s, a) + \gamma \sum_{s'} P(s, a, s')V(s')\right)
\end{equation*}

- V = Value
- S = current state
- a = Action
- R = Reward
- gamma = discount factor
- s' = prime means next state

Completley non-deterministic - the actions we take happen 80% of the time...

The value of the square is the current rewards plus the sum of the probablility of all actions and next states times their value times gamma the discount factor.

### Value Iteration Algorithm

- Initialise a table V of value estimates for each square with all zeros
- Loop over every possible state s
    - From state s loop over every possible action a
        - Get a list of all (probability, reward, s') transition tuples from state s, action a
        - expected_reward = sum of all possibel rewards multiplied by the probabilities
        - expected_value = loopup V[s'] foro each possible s', multiply by probaility, sum
        - action_value = expected_reward + gamme * expected_valye
    - set V[s] to best action_value_found
    
https://github.com/colinskow/move37/blob/master/dynamic_programming/grid_world.py
http://localhost:8888/notebooks/Gridworld-with-Q-Learning-Reinforcement-Learning-/Gridworld.ipynb

In [60]:
def best_action_value(grid, V, state):
    
    v_max = float('-inf')
    a_max = None

    for action, next_state in grid.get_valid_moves(state):
        transition_probabilities = grid.transition_probabilities(action, state)
        expected_v = 0
        expected_r = 0
        v = 0
        
        # Calculate the summed expected value and reward
        for (prob, r, next_state) in transition_probabilities:
            expected_v += prob * V[next_state[0], next_state[1]]
            expected_r += prob * r
        
        # Add gamma as discount factor (we care less about future reward than reward now.)
        v = expected_r + 0.9 * expected_v

        if v > v_max:
            v_max = v
            a_max = action
    
    return a_max, v_max

In [74]:
def calculate_values(grid, policy):
    max_change = float('inf')

    while max_change > 0.0001:
        new_policy = np.zeros((g.height,g.width))
        for state in grid.available_states:
            a_max, v_max = best_action_value(grid, policy, state)

            new_policy[state[0], state[1]] = v_max
        

        max_change = np.max(policy - new_policy)
        policy = new_policy

    return policy

g = Gridworld(DEFAULT)
policy = np.zeros((g.height,g.width))
calculate_values(g, policy)

array([[ 0.20197319,  0.45434235,  0.67095015,  0.90658975,  0.        ],
       [-0.04462392,  0.        ,  0.        ,  0.50668304,  0.        ],
       [-0.27999523, -0.12936727,  0.07084305,  0.25499121, -0.02825949]])

- https://itnext.io/reinforcement-learning-with-q-tables-5f11168862c8
- https://towardsdatascience.com/advanced-dqns-playing-pac-man-with-deep-reinforcement-learning-3ffbd99e0814
- https://towardsdatascience.com/advanced-dqns-playing-pac-man-with-deep-reinforcement-learning-3ffbd99e0814
- https://medium.com/free-code-camp/how-to-apply-reinforcement-learning-to-real-life-planning-problems-90f8fa3dc0c5
- https://towardsdatascience.com/self-learning-ai-agents-part-ii-deep-q-learning-b5ac60c3f47
- https://medium.com/@zsalloum/monte-carlo-in-reinforcement-learning-the-easy-way-564c53010511

Good Blog Example...
https://towardsdatascience.com/reinforcement-learning-rl-101-with-python-e1aa0d37d43b